## Finding the Center of Mass of the Police Districts

* In this notebook, we find the centers of mass for each of the police districts, represented by polygons.

In [1]:
import os
import time
from tqdm import tqdm

import gmaps
import shapely
import numpy as np
import pandas as pd
import geopandas as gpd

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Set up API Authentication.
key_path = "/Users/administrator/Documents/Projects/abq_crime/api_key.txt"
with open(key_path) as f:
    api_key = f.readline()
    f.close()

# Configure Google Maps API.
gmaps.configure(api_key=api_key)

In [3]:
shapefile_path = "/Users/administrator/Documents/Projects/sf-crime-exploration/data/PdDistricts"
districts_dataframe = gpd.read_file(shapefile_path)
districts_dataframe = districts_dataframe[["district", "geometry"]]

### Splitting MultiPolygons

* In this section we split up multipolygons that represent a single police district. 
* We add the new polygons as entries in the dataframe and delete the original one.

In [4]:
# Find the indices of entries that contain a multipolygon in the geometry column.
mpolygon_indices = [type(item) == shapely.geometry.multipolygon.MultiPolygon for item in districts_dataframe["geometry"]]
mpolygon_indices = [idx for idx, item in enumerate(mpolygon_indices) if item == True]

In [5]:
# For the entries that contain a multipolygon, split up the polygons.
new_entries = list()
for index in mpolygon_indices:
    district = districts_dataframe.iloc[index]["district"]
    for poly in districts_dataframe.iloc[index]["geometry"]:
        new_entries.append((district, poly))

In [6]:
# Add these new entries into the dataframe, and delete the original ones.
for entry in new_entries:
    districts_dataframe.loc[len(districts_dataframe)] = entry

# Delete the original entries.
districts_dataframe.drop(mpolygon_indices, axis=0, inplace=True)
districts_dataframe.reset_index(drop=True, inplace=True)

### Converting from (Longitude, Latitude) to (Latitude, Longitude)

* In this section we convert the vertices of the polygons from the (longitude, latitude) convention to the (latitude, longitude) convention.
* This conversion is done so that the rest of the notebook is consistent and because most functions assume the (latitude, longitude) convention, especially the Google Maps API.

In [7]:
def convert_coordinates(polygon):
    """Converts coordinates from (longitude, latitude) to (latitude, longitude) for a collection of points.

    Parameters
    ----------
    polygon : shapely.geometry.Polygon instance
        Shapely Polygon containing coordinates in (longitude, latitude) format.

    Returns
    -------
    polygon : shapely.geometry.Polygon instance
        Shapely Polygon containing coordinates are in (latitude, longitude) format.

    """
    polygon_vertices = polygon.exterior.coords[:]
    for idx, vertex in enumerate(polygon_vertices):
        polygon_vertices[idx] = vertex[::-1]
    polygon = shapely.geometry.Polygon(polygon_vertices)

    return polygon

In [8]:
# Convert the coordinates from (longitude, latitude) to (latitude, longitude).
districts_dataframe["geometry"] = districts_dataframe["geometry"].apply(lambda x: convert_coordinates(x))

### Computing the Centers of Mass

* In this section, we compute the center of mass for each of the polygons representing the police districts.

As a test, let's find the center of mass for one of the polygons.

In [9]:
test_polygon = districts_dataframe.iloc[0]["geometry"]
centroid_x = test_polygon.centroid.xy[:][0][0]
centroid_y = test_polygon.centroid.xy[:][1][0]

In [10]:
# Plot the polygon and the centroid using Google Maps.
fig = gmaps.figure()
drawing = gmaps.drawing_layer([gmaps.Polygon(test_polygon.exterior.coords[:], stroke_color="black", stroke_opacity=0.2,)])
marker = gmaps.marker_layer([(centroid_x, centroid_y)]) 
fig.add_layer(drawing)
fig.add_layer(marker)
fig

Figure(layout=FigureLayout(height='420px'))

From the above results, it looks like we can proceed. To modify the above method, we just need to do this for all the geometries and use lists instead of just a single marker/drawing.

In [11]:
# Collate the district polygons into a single list. Each element in this list will be a shapely.geometry.Polygon instance.
district_polygons = [districts_dataframe["geometry"][idx] for idx in range(len(districts_dataframe))]
district_centroids = [polygon.centroid.xy[:] for polygon in district_polygons]
district_vertices = [polygon.exterior.coords[:] for polygon in district_polygons]

In [12]:
# Populate the drawing and marker lists.
drawing_list = list()
marker_list = list()

for idx, vertices in enumerate(district_vertices):
    drawing_list.append(gmaps.Polygon(vertices, stroke_color="black", stroke_opacity=0.2,))
    centroid = district_centroids[idx]
    marker_list.append((centroid[0][0], centroid[1][0]))

In [13]:
# Plot the Polygons using the Google Maps API.
fig = gmaps.figure()
drawings = gmaps.drawing_layer(drawing_list)
markers = gmaps.marker_layer(marker_list)
fig.add_layer(drawings)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

The centroids of the police districts have been computed. However, we should keep in mind that the Southern police district is composed of two polygons, so we really only have to pick one.

### Export the Centroid Information to CSV

* With the centroids computed we can store the information in a Pandas DataFrame object for future usage.
* Note that the new CSV file assumes (latitude, longitude) instead of (longitude, latitude).

First, we need to edit the centroids information so that it is more easily accessible.

In [14]:
centroids_list = list()
for centroid in district_centroids:
    centroids_list.append(shapely.geometry.Point(centroid[0][0], centroid[1][0]))
print(centroids_list)

[<shapely.geometry.point.Point object at 0x7fc11bf05290>, <shapely.geometry.point.Point object at 0x7fc11bf058d0>, <shapely.geometry.point.Point object at 0x7fc11bf05210>, <shapely.geometry.point.Point object at 0x7fc11bf05890>, <shapely.geometry.point.Point object at 0x7fc11bf05910>, <shapely.geometry.point.Point object at 0x7fc11bf05450>, <shapely.geometry.point.Point object at 0x7fc11bf05f50>, <shapely.geometry.point.Point object at 0x7fc11bf052d0>, <shapely.geometry.point.Point object at 0x7fc11bf05250>, <shapely.geometry.point.Point object at 0x7fc11bf05650>, <shapely.geometry.point.Point object at 0x7fc11bf054d0>]


In [51]:
# Create a new DataFrame object containing the district and centroid information ONLY.
# Make sure to use GeoPandas!
centroids_dataframe = districts_dataframe.copy()
centroids_dataframe = centroids_dataframe.assign(centroid=centroids_list)


AttributeError: module 'geopandas' has no attribute 'assign'

You left off above. The issue is that you need to use GeoPandas instead of Pandas DataFrame because that does not preserve the Shapely geometry information.

In [36]:
# Save the DataFrame into a CSV file.
path = "/Users/administrator/Documents/Projects/sf-crime-exploration/data/SFPD_District_Centroids.csv"
centroids_dataframe.to_csv(path, index=False)

### Load Data from CSV File

* In this section we load the CSV file that was just created, to ensure that the data is accessible.

In [37]:
centroids_dataframe = pd.read_csv(path)
centroids_dataframe

,district,geometry,centroid
0,BAYVIEW,POLYGON ((37.76480022019017 -122.3809828136006...,POINT (37.73432830504337 -122.3896413590277)
1,MISSION,"POLYGON ((37.769317718404 -122.4095391743523, ...",POINT (37.75756983991986 -122.4226455774392)
2,NORTHERN,POLYGON ((37.80793010706978 -122.4337921715291...,POINT (37.78998465312932 -122.4317576421874)
3,TENDERLOIN,"POLYGON ((37.7862601311238 -122.4021713308171,...",POINT (37.78236656943267 -122.412761876948)
4,CENTRAL,"POLYGON ((37.80683854001055 -122.426120390961,...",POINT (37.79791152508096 -122.4091616666903)
5,PARK,"POLYGON ((37.7831382837805 -122.4395563094215,...",POINT (37.76434908040622 -122.4491077189602)
6,RICHMOND,"POLYGON ((37.79148984438553 -122.441268713802,...",POINT (37.77760526628989 -122.4796602993393)
7,INGLESIDE,POLYGON ((37.74858097286839 -122.4044981875426...,POINT (37.72788335066097 -122.4315859501734)
8,TARAVAL,POLYGON ((37.70810461000542 -122.4984166706208...,POINT (37.73663284365935 -122.4818302825414)
9,SOUTHERN,POLYGON ((37.79424680970543 -122.3918613881226...,POINT (37.77632134223039 -122.3988620645398)


In [38]:
# Ensure that the type of object in the geometry column is a shapely.geometry.Polygon instance.
type(centroids_dataframe["geometry"][0])

str

For some odd reason the data in the geometry column keeps getting saved as a str object instead of a shapely.geometry.Polygon object.

Let's examine the original DataFrame.

In [20]:
type(districts_dataframe["geometry"][0])

shapely.geometry.polygon.Polygon

Interesting since the original DataFrame's geometry column contains shapely.geometry.Polygon objects.